In [29]:
# First attempt to create network to recognize 3 different people (Jackie Chan, Barack Obama and Rachel McAdams) v1.0
# Manual loading images
# No image augmentation, no generator
# "Random" network structure based on one of articles from http://machinelearningmastery.com/
# Not so bad results - 70% accuracy on the "basic" set

In [30]:
#all imports
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
#import cv2
import csv
import imageio

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K


In [31]:
K.set_image_dim_ordering('th')
seed = 7
np.random.seed(seed)

In [32]:
def rgb2gray(rgb):

    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

def readimagesfromfolder(path, color, show):
	imagepackage_list = []
	imagepackage = []
	imagelist = []
	filelist = sorted(os.listdir(path))
	for filename in filelist:
		fullfilename =  path + filename
		#print(fullfilename)        
		singleimage = imageio.imread(fullfilename)
		singleimage = rgb2gray(singleimage)
		#print(singleimage)        
		if show==1:
			plt.imshow(singleimage)
			plt.show()
		imagepackage.append(singleimage)
		imagepackage_list.append(imagepackage)        
		imagepackage = []        
	#print(len(imagepackage))        
	imagepackage_list = np.float32(imagepackage_list)
	#cv2.destroyAllWindows()
	return imagepackage_list

def readlabelfromcsv(filename):
    training_y = []
    with open(filename) as f:
        #training_y.append(f.readline())
        content = csv.reader(f)
        for line in content:
            #print(line[0])
            training_y.append(int(line[0]))
    return training_y




In [33]:
       
folderpath = 'data/basic/training/'
X_train=readimagesfromfolder(folderpath, 0, 0)
print(np.shape(X_train))

folderpath = 'data/basic/test/'
X_test=readimagesfromfolder(folderpath, 0, 0)
print(np.shape(X_test))

y_train = readlabelfromcsv("data/basic/training.csv")
y_test = readlabelfromcsv("data/basic/test.csv")

print(y_train)
print(np.shape(y_train))

print(y_test)
print(np.shape(y_test))


(162, 1, 180, 180)
(48, 1, 180, 180)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
(162,)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
(48,)


In [34]:
# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [35]:
def larger_model():
	# create model
	model = Sequential()
	model.add(Convolution2D(8, 5, 5, border_mode='valid', input_shape=(1, 180, 180), activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Convolution2D(16, 3, 3, activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.5))
	model.add(Flatten())
	model.add(Dense(128, activation='relu'))
	model.add(Dense(50, activation='relu'))
	model.add(Dense(num_classes, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

# build the model
model = larger_model()
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=50, batch_size=32, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

Train on 162 samples, validate on 48 samples
Epoch 1/50
0s - loss: 1.4956 - acc: 0.2593 - val_loss: 1.2411 - val_acc: 0.3750
Epoch 2/50
0s - loss: 1.2151 - acc: 0.3025 - val_loss: 1.1200 - val_acc: 0.3125
Epoch 3/50
0s - loss: 1.0630 - acc: 0.4691 - val_loss: 1.0693 - val_acc: 0.5208
Epoch 4/50
0s - loss: 0.9745 - acc: 0.6481 - val_loss: 1.0133 - val_acc: 0.4375
Epoch 5/50
0s - loss: 0.8308 - acc: 0.7037 - val_loss: 0.9227 - val_acc: 0.5417
Epoch 6/50
0s - loss: 0.6597 - acc: 0.7778 - val_loss: 0.9105 - val_acc: 0.5625
Epoch 7/50
0s - loss: 0.5586 - acc: 0.7963 - val_loss: 0.8354 - val_acc: 0.5833
Epoch 8/50
0s - loss: 0.3694 - acc: 0.8642 - val_loss: 0.7576 - val_acc: 0.5833
Epoch 9/50
0s - loss: 0.2822 - acc: 0.9074 - val_loss: 0.8883 - val_acc: 0.6250
Epoch 10/50
0s - loss: 0.2131 - acc: 0.9444 - val_loss: 0.7648 - val_acc: 0.6250
Epoch 11/50
0s - loss: 0.1271 - acc: 0.9691 - val_loss: 0.7688 - val_acc: 0.6250
Epoch 12/50
0s - loss: 0.0842 - acc: 0.9938 - val_loss: 0.7975 - val_acc: